In [100]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import yfinance as yf

## Importing Our List of Stocks
Imported the names of S&P companies. Many companies in the S&P 500 have multiple stocks so some of them aren't found down the line.

In [101]:
stocks = pd.read_csv('sp_500_stocks_new.csv')

## Using Y-finance
* Market capitalization for each stock
* Price of each stock



In [102]:
symbol = 'aapl'
data = yf.Ticker(symbol)
#Historical prices 

In [103]:
price = data.info['currentPrice']
marketcap = data.info['marketCap']
price

189.87

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [104]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization' , 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

In [105]:
new_data = pd.Series([symbol, price, marketcap,'N/A'], index= my_columns)

In [106]:
final_dataframe = pd.concat([final_dataframe, new_data.to_frame().T], ignore_index=True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,aapl,189.87,2911485558784,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [107]:
final_dataframe = pd.DataFrame(columns = my_columns)
counter = 0
for stock in stocks['Ticker']:
    symbol = stock
    try:
        data = yf.Ticker(symbol)
        price = data.info['currentPrice']
        marketcap = data.info['marketCap']
        new_data = pd.Series([symbol, price, marketcap,'N/A'], index= my_columns)
        final_dataframe = pd.concat([final_dataframe, new_data.to_frame().T], ignore_index=True)
    except:
        print(stock,' not found')
        counter += 1    

BRK.B  not found
BF.B  not found


In [108]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,105.26,58246782976,N/A
1,AOS,85.68,12566685696,N/A
2,ABT,104.09,181078081536,N/A
3,ABBV,166.42,293876072448,N/A
4,ACN,303.59,190875844608,N/A
...,...,...,...,...
498,ZBRA,317.89,16345713664,N/A
499,ZBH,120.18,24724391936,N/A
500,ZTS,174.06,79422709760,N/A
501,MMM,105.26,58246782976,N/A


## Using Batch API Calls to Improve Performance
Trying to use batch api

In [109]:
def chunks(lst, n):
    for i in range(0, len(lst) , n):
        yield lst[i:i+n]

In [111]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = yf.Ticker(symbol_string)
    for symbol in symbol_string.split(','):
        try:
            data = yf.Ticker(symbol)
            price = data.info['currentPrice']
            marketcap = data.info['marketCap']
            new_data = pd.Series([symbol, price, marketcap,'N/A'], index= my_columns)
            final_dataframe = pd.concat([final_dataframe, new_data.to_frame().T], ignore_index=True)
        except:
            print(symbol,' not found')

BRK.B  not found
BF.B  not found


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [46]:
while True:
    portfolio_size = input('Enter the value of your portfolio: ')
    try:
        portfolio_size = int(portfolio_size)
        break
    except ValueError:
        print('Invalid input. Enter an integer please. Try Again.')
        

val = float(portfolio_size)

Enter the value of your portfolio:  10_000_000


In [57]:
position_size = val / len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Stock Price'])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,105.21,58263326720,188
1,AOS,85.49,12472084480,232
2,ABT,104.41,181443411968,190
3,ABBV,165.39,293416960000,120
4,ACN,309.74,189234872320,64
...,...,...,...,...
498,ZBRA,320.74,16405103616,61
499,ZBH,121.3,24666787840,163
500,ZTS,174.05,79618916352,114
501,MMM,105.21,58274447360,188


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [94]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, sheet_name = 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [95]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format({
    'font_color':font_color,
    'bg_color':background_color,
    'border':1
}
)
dollar_format = writer.book.add_format({
    'num_format': '$0.00',
    'font_color':font_color,
    'bg_color':background_color,
    'border':1
}
)
integer_format = writer.book.add_format({
    'num_format':'0',
    'font_color':font_color,
    'bg_color':background_color,
    'border':1
}
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [96]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18,string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18,string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18,string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18,string_format)
# writer.close()
# writer.sheets['Recommended Trades'].write('A1','Ticker', string_format)
# # writer.sheets['Recommended Trades'].write('B1','Stock Price', dollar_format)
# writer.sheets['Recommended Trades'].write('C1','Market Capitalization', dollar_format)
# writer.sheets['Recommended Trades'].write('D1','Number of Shares to Buy', integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [99]:
columns_formats = {
    'A':['Ticker', string_format],
    'B':['Stock Price', dollar_format],
    'C':['Market Capitalization', dollar_format],
    'D':['Number of Shares to Buy', integer_format]
}

for column in columns_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, columns_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1',columns_formats[column][0] ,columns_formats[column][1])
    
writer.close()